In [33]:
import csv
import heapq
from datetime import datetime
from datetime import timedelta
import pandas as pd

# Step 1: Parse the CSV file
def parse_schedule_csv(file_path):
    with open(file_path, mode='r') as csvfile:
        reader = csv.DictReader(csvfile)
        schedule_data = [row for row in reader]
    return schedule_data

In [34]:
def time_to_minutes(time_str):
    # A helper function to convert time in HH:MM:SS format to minutes
    hours, minutes, _ = map(int, time_str[1:-1].split(':'))
    return hours * 60 + minutes

In [35]:
def build_graph(schedule_data):
    graph = {}
    added_minutes = 0  # To track added time due to next day arrival

    for i in range(len(schedule_data) - 1):
        train_no = schedule_data[i]['Train No.'].strip()
        from_station = schedule_data[i]['station Code'].strip()
        from_islno = schedule_data[i]['islno'].strip()
        departure_time = schedule_data[i]['Departure time'].strip()
        distance = int(schedule_data[i]['Distance'])

        # Reset added_minutes if we are looking at a new train
        if i == 0 or schedule_data[i - 1]['Train No.'].strip() != train_no:
            added_minutes = 0

        departure_minutes = time_to_minutes(departure_time) + added_minutes

        # Check the next entry only if it is for the same train
        if schedule_data[i + 1]['Train No.'].strip() == train_no:
            to_station = schedule_data[i + 1]['station Code'].strip()
            to_islno = schedule_data[i + 1]['islno'].strip()
            arrival_time = schedule_data[i + 1]['Arrival time'].strip()
            next_distance = int(schedule_data[i + 1]['Distance'])

            arrival_minutes = time_to_minutes(arrival_time) + added_minutes

            # Check if the arrival time is on the next day
            if arrival_minutes < departure_minutes:
                arrival_minutes += 1440  # Add 24 hours in minutes
                added_minutes += 1440   # Carry this forward for subsequent stops

            weight = next_distance - distance

            edge_data = {
                'to_station': to_station,
                'to_islno': to_islno,
                'train_no': train_no[1:-1],
                'from_islno': from_islno,
                'departure_minutes': departure_minutes,
                'arrival_minutes': arrival_minutes,
                'departure': departure_time[1:-1],
                'arrival': arrival_time[1:-1],
                'weight': weight
            }

            if from_station not in graph:
                graph[from_station] = []
            graph[from_station].append(edge_data)

    return graph


In [36]:
def calculate_cost(from_station, to_station, connection, cost_type, graph,new_day,train_changed):
    # The connection is now a dictionary with time and distance information
    if cost_type == 'stops':
        # Assuming 1 stop per connection here
        return 1
    elif cost_type == 'distance':
        # Distance is already provided as the weight in the graph
        return connection['weight']
    elif cost_type == 'price':
        if train_changed==0 and new_day==0:
            x=0
        if train_changed==0 and new_day==1:
            x=1
        if train_changed==1 and new_day==1:
            x=1
        if train_changed==1 and new_day==0:
            x=1
        #print("inside of cost",train_changed, new_day,x) 
        return x


    elif cost_type[0:11] == 'arrivaltime':
        # Parse the start time and the edge's departure and arrival times
        start_datetime = datetime.strptime(str(cost_type[12:]), "%H:%M:%S")
        departure_datetime = datetime.strptime(connection['departure'], "%H:%M:%S")
        arrival_datetime = datetime.strptime(connection['arrival'], "%H:%M:%S")
        
        if departure_datetime < start_datetime:
            departure_datetime += timedelta(days=1)
        if arrival_datetime < departure_datetime:
            arrival_datetime += timedelta(days=1)

        total_duration = arrival_datetime - start_datetime
        return total_duration
    
    else:
        raise ValueError("Unknown cost type")


In [37]:
def dijkstra(graph, start, end, cost_function):
    if cost_function[0:11] == 'arrivaltime':
         queue = [(timedelta(days=0, hours=0, minutes=0, seconds=0), start, [],None,[],None,[], datetime.strptime(cost_function[12:], "%H:%M:%S"))]
    else:
        queue = [(0, start, [],None,[],None,[], datetime.strptime('00:00:00', "%H:%M:%S"))]  # (cost, current_station, path,train no, from station no, arrival_time_at_current_station)
    visited = set()
    is_first_iteration=True
    while queue:
        (cost, current_station, path, train, train_list, from_station,from_station_list, arrival_time_at_current_station) = heapq.heappop(queue)
        if current_station not in visited:
            visited.add(current_station)
            path = path + [current_station]
            train_list=train_list+[train]
            from_station_list=from_station_list+[from_station]

            if current_station == end:
                return (cost, path, train_list[1:],from_station_list[1:])  # Return the cost and path as a tuple
            if cost_function[0:11] == 'arrivaltime':
                cost_function=cost_function.replace(cost_function[12:], arrival_time_at_current_station.strftime("%H:%M:%S"))
            for edge_data in graph.get(current_station, []):
                next_station = edge_data['to_station']
                next_train_no=edge_data['train_no']
                from_station_no=edge_data['from_islno']

                if next_station not in visited:
                    departure_time = datetime.strptime(edge_data['departure'], "%H:%M:%S") ### this is next possible connection!!!!
                    arrival_time = datetime.strptime(edge_data['arrival'], "%H:%M:%S") 

                    # Check if the arrival time at the current station is before the departure time of the next train
                    new_day=0
                    if departure_time.hour*60+departure_time.minute>arrival_time.hour*60+arrival_time.minute:
                        new_day=1

                    if departure_time.hour*60+departure_time.minute<arrival_time_at_current_station.hour*60+arrival_time_at_current_station.minute:
                            departure_time=departure_time+timedelta(minutes=1440)
                
                    train_changed=0
                    ###check if the trains are switched
                    if next_train_no!=train:
                        train_changed=1

                    time_difference=departure_time-arrival_time_at_current_station

                    if is_first_iteration==True or train_changed==0:
                        total_cost = cost + calculate_cost(current_station,next_station,edge_data,cost_function,graph,new_day,train_changed)
                        new_path = path.copy()  # Make a copy of the current path
                        new_train_list=train_list.copy()
                        new_from_station_list=from_station_list.copy()
                        heapq.heappush(queue, (total_cost, next_station, new_path,next_train_no,new_train_list,from_station_no,new_from_station_list, datetime.strptime(edge_data['arrival'], "%H:%M:%S")))

                    else: 
                        if time_difference.total_seconds()/60>=10: 
                            total_cost = cost + calculate_cost(current_station,next_station,edge_data,cost_function,graph,new_day,train_changed)
                            new_path = path.copy()  # Make a copy of the current path
                            new_train_list=train_list.copy()
                            new_from_station_list=from_station_list.copy()
                            heapq.heappush(queue, (total_cost, next_station, new_path,next_train_no,new_train_list,from_station_no,new_from_station_list, datetime.strptime(edge_data['arrival'], "%H:%M:%S")))
            is_first_iteration=False
    return (float('inf'), [],[],[])  # Return infinity and an empty path if the destination is not reachable


In [38]:
def get_connection(path,train_list,from_station_list):
    result=""
    visited=set()
    for i in range(len(train_list)):
        if i==len(train_list)-1:
            if str(train_list[i]) not in visited:
               result=result+str(train_list[i]) +" : "+str(from_station_list[i]) + " -> " +str(int(from_station_list[i])+1)
            else:
                result=result +str(int(from_station_list[i])+1) + " ; "
        else:
            if str(train_list[i]) not in visited and (train_list[i]!=train_list[i+1]) :
                result=result+str(train_list[i]) +" : "+str(from_station_list[i]) + " -> " +str(int(from_station_list[i])+1) + " ; "
                visited.add(str(train_list[i]))
            elif str(train_list[i]) not in visited and (train_list[i]==train_list[i+1]) :
                result=result+str(train_list[i]) +" : "+str(from_station_list[i]) + " -> "
                visited.add(str(train_list[i]))
            elif str(train_list[i]) in visited and str(train_list[i+1]) not in visited:
                result=result +str(int(from_station_list[i])+1) + " ; "
    #print(result)
    return result

In [39]:
# Example usage
schedule_data = parse_schedule_csv('schedule.csv')
graph = build_graph(schedule_data)
cost, path,train_list,from_station_list = dijkstra(graph, 'DPW', 'GGM', 'stops')
print(f"Cost: {cost}, Path: {path}")
get_connection(path,train_list,from_station_list)


Cost: 18, Path: ['DPW', 'SMC', 'CRN', 'LTA', 'NWA', 'NIR', 'PDE', 'GNS', 'SKY', 'BUQ', 'HBG', 'JBP', 'BSL', 'BPL', 'PUNE', 'BRC', 'ADI', 'SAU', 'GGM']


'58865 : 2 -> 3 ; 10001 : 2 -> 12 ; 12142 : 7 -> 8 ; 12107 : 5 -> 6 ; 12630 : 3 -> 4 ; 22695 : 4 -> 5 ; 12298 : 4 -> 5 ; 19017 : 18 -> 19 ; 59548 : 43 -> 44'

In [40]:
# Example usage
schedule_data = parse_schedule_csv('schedule.csv')
graph = build_graph(schedule_data)
cost, path,train_list,from_station_list = dijkstra(graph, 'ARK', 'BARH', 'distance')
print(f"Cost: {cost}, Path: {path}")
get_connection(path,train_list,from_station_list)


Cost: 1255, Path: ['ARK', 'GPJ', 'DPC', 'PFU', 'BHJA', 'MKRD', 'PBV', 'SUKU', 'KRPU', 'DMNJ', 'LKMR', 'TKRI', 'RUL', 'LLGM', 'BLMK', 'SKPI', 'KTGA', 'SPRD', 'THV', 'MNGD', 'KSNG', 'TIG', 'BLGR', 'BRGA', 'SBP', 'JSG', 'BMB', 'GP', 'KXN', 'KLG', 'PPO', 'ROU', 'BZR', 'BUL', 'JRA', 'MOU', 'GOL', 'SWR', 'CKP', 'RKSN', 'SINI', 'KND', 'CNI', 'BBM', 'PRR', 'ANR', 'JOC', 'ASN', 'CRJ', 'VDS', 'MDP', 'JSME', 'STL', 'JAJ', 'GHR', 'JMU', 'MNP', 'BSQP', 'KIUL', 'BRYA', 'HTZ', 'MKA', 'BARH']


'58501 : 16 -> 24 ; 18006 : 3 -> 6 ; 18448 : 11 -> 17 ; 12807 : 8 -> 10 ; 12375 : 16 -> 17 ; 08302 : 22 -> 22 ; 18029 : 61 -> 62 ; 13287 : 14 -> 15 ; 18418 : 15 -> 16 ; 33 ; 18478 : 56 -> 57 ; 58114 : 41 -> 42 ; 58112 : 89 -> 90 ; 67 ; 12872 : 17 -> 18 ; 23 ; 58012 : 5 -> 6 ; 28 ; 12865 : 9 -> 10 ; 13302 : 4 -> 5 ; 31 ; 02511 : 13 -> 14 ; 08449 : 8 -> 9 ; 15027 : 14 -> 15 ; 18449 : 19 -> 20 ; 22843 : 14 -> 15 ; 13007 : 14 -> 15 ; 83131 : 6 -> 7 ; 9 ; 12351 : 10 -> 13 ; 13413 : 14 -> 15 ; 03401 : 17 -> 18 ; 03235 : 17 -> 19 ; '

In [41]:
# Example usage
schedule_data = parse_schedule_csv('schedule.csv')
graph = build_graph(schedule_data)
cost, path,train_list,from_station_list = dijkstra(graph, 'MKRH', 'RC', 'price')
print(f"Cost: {cost}, Path: {path}")
get_connection(path,train_list,from_station_list)


Cost: 5, Path: ['MKRH', 'RMPB', 'MLNH', 'GRMP', 'BNDP', 'DLPH', 'DOTL', 'MANG', 'GZO', 'EKI', 'ADF', 'OMLF', 'MLDT', 'RPH', 'HWH', 'KGP', 'BHC', 'BBS', 'VSKP', 'BZA', 'SC', 'SEM', 'RC']


'23154 : 2 -> 14 ; 02510 : 6 -> 11 ; 02853 : 3 -> 5 ; 02882 : 4 -> 5 ; 22692 : 6 -> 8 ; '

In [42]:
# Example usage
schedule_data = parse_schedule_csv('schedule.csv')
graph = build_graph(schedule_data)
cost, path,train_list,from_station_list = dijkstra(graph, 'ASV', 'MWT', 'arrivaltime 10:44:00')
print(f"Cost: {cost}, Path: {path}")
get_connection(path,train_list,from_station_list)


Cost: 1 day, 7:48:00, Path: ['ASV', 'ADI', 'SBI', 'KLL', 'MSH', 'UJA', 'SID', 'CHP', 'PNU', 'SIM', 'ABR', 'SRPJ', 'SOH', 'NANA', 'MOI', 'JWB', 'FA', 'MJ', 'PMY', 'LUNI', 'JU', 'RKB', 'MMY', 'TIW', 'OSN', 'MWT']


'19943 : 8 -> 9 ; 19707 : 13 -> 28 ; 19403 : 6 -> 7 ; 16312 : 37 -> 40 ; 04801 : 1 -> 6 ; '

In [44]:
def read_and_solve_problems(input_file_path,output_file_path):
    df = pd.read_csv(input_file_path)
    solutions = pd.DataFrame()
    solutions = pd.DataFrame(columns=['ProblemNo', 'Connection', 'Cost'])

    for i in range(len(df)):
        #reading problems
        ProblemNo=df['ProblemNo'][i]
        FromStation=df['FromStation'][i]
        ToStation=df['ToStation'][i]
        Schedule=df['Schedule'][i]
        CostFunction=df['CostFunction'][i]
        #running search algorithm
        schedule_data = parse_schedule_csv(Schedule)
        graph = build_graph(schedule_data)
        cost, path,train_list,from_station_list = dijkstra(graph, FromStation, ToStation, CostFunction)
        result=get_connection(path,train_list,from_station_list)
        #writing solutions
        solutions.loc[i, 'ProblemNo'] = ProblemNo
        solutions.loc[i, 'Connection'] = result

        #solutions = solutions.append({'ProblemNo': ProblemNo, 'Connection': result, 'Cost': cost}, ignore_index=True)
        if CostFunction[:11]=='arrivaltime':
            datetime1 = datetime.strptime(CostFunction[12:], "%H:%M:%S")
            datetime2 = datetime1 + cost

            # Extract days, hours, minutes, and seconds
            days = datetime2.day-1
            hours=datetime2.hour
            minutes=datetime2.minute
            seconds=datetime2.second

            # Format as DD HH MM SS
            if days==0:
                formatted_duration = f"{hours:02d}:{minutes:02d}:{seconds:02d}"
            else:
                formatted_duration = f"{days:02d}:{hours:02d}:{minutes:02d}:{seconds:02d}"
            solutions.loc[i, 'Cost'] = formatted_duration
        else:
            solutions.loc[i, 'Cost'] = cost

        solutions.to_csv(output_file_path, index=False)



In [45]:
read_and_solve_problems('problems.csv','solutions.csv')